## 医学論文の自動仕分けチャレンジ 推論1

In [1]:
import os
import sys

#from google.colab import drive
#drive.mount('/gdrive')

#!cp /gdrive/MyDrive/Datasets/signate-471/train.csv .
#!cp /gdrive/MyDrive/Datasets/signate-471/test.csv .
#!cp /gdrive/MyDrive/Datasets/signate-471/sample_submit.csv .

In [2]:
#!pip install -q transformers

In [3]:
import math
import random
import time
import warnings

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import transformers as T
from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

from transformers import AutoTokenizer, AutoModel


In [5]:
class CFG:
    batch_size = 5 #16
    num_workers = 3 #4
    max_length =  512 #256 #72
    n_splits = 5
    version = 119
    model = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext" 
    
    epochs = 4

In [6]:
DATA_DIR = "../input/"
OUTPUT_DIR = "../output/"

In [7]:
warnings.filterwarnings("ignore")

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = 'cpu'

In [9]:
def init_logger(log_file=OUTPUT_DIR + "train.log"):
    from logging import INFO, FileHandler, Formatter, StreamHandler, getLogger

    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()

In [10]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed = 471
seed_torch(seed)

## データ読み込み

In [11]:
train = pd.read_csv(DATA_DIR + "train.csv")
test = pd.read_csv(DATA_DIR + "test.csv")
sub = pd.read_csv(DATA_DIR + "sample_submit.csv", header=None)
sub.columns = ["id", "judgement"]
TARGET = "judgement"

In [12]:
# この値を境に、モデルの出力を 0 と 1 にします。
border = len(train[train[TARGET] == 1]) / len(train[TARGET])
print(border)
init_border = border

0.023282372444280715


## 前処理

In [13]:
 # preprocess
train["text"] = train["title"] + " " + train["abstract"].fillna("")
test["text"] = test["title"] + " " + test["abstract"].fillna("")

In [14]:
def get_train_data(train):

    # 交差検証 用の番号を振ります。
    Fold = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=seed) #5
    for n, (train_index, val_index) in enumerate(Fold.split(train, train[TARGET])):
        train.loc[val_index, "fold"] = int(n)
    train["fold"] = train["fold"].astype(np.uint8)

    return train

In [15]:
def get_test_data(test):
    return test

In [16]:
train = get_train_data(train)

## データセット定義

In [21]:
class BaseDataset(Dataset):
    def __init__(self, df, model_name, include_labels=True):
        #tokenizer = T.BertTokenizer.from_pretrained(model_name)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.df = df
        self.include_labels = include_labels

        self.title = df["text"].tolist()
        self.encoded = tokenizer.batch_encode_plus(
            self.title,
            padding = 'max_length',            
            max_length = CFG.max_length,
            truncation = True,
            return_attention_mask=True
        )
        
        if self.include_labels:
            self.labels = df[TARGET].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        input_ids = torch.tensor(self.encoded['input_ids'][idx])
        attention_mask = torch.tensor(self.encoded['attention_mask'][idx])

        if self.include_labels:
            label = torch.tensor(self.labels[idx]).float()
            return input_ids, attention_mask, label

        return input_ids, attention_mask

## モデル定義

In [22]:
class BaseModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        #self.model = AutoModel.from_pretrained(model_name)
        self.model = T.BertForSequenceClassification.from_pretrained(model_name, num_labels=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        out = self.model(input_ids=input_ids, attention_mask=attention_mask) #,labels=labels)
        logits = out.logits
        #out = self.sigmoid(out.logits).squeeze()
        out = self.sigmoid(logits).squeeze()
        #out = logits.squeeze()
        return out

## ツール

In [23]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))

## 学習補助関数

In [24]:
def train_fn(train_loader, model, criterion, optimizer, epoch, device):
    start = end = time.time()
    losses = AverageMeter()
    
    # switch to train mode
    model.train()
    
    for step, (input_ids, attention_mask, labels) in enumerate(train_loader):
        
        optimizer.zero_grad()
        
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        
        batch_size = labels.size(0)
        
        y_preds = model(input_ids, attention_mask)
        
        loss = criterion(y_preds, labels)
        
        # record loss
        losses.update(loss.item(), batch_size)
        loss.backward()
        
        optimizer.step()
        
        if step % 100 == 0 or step == (len(train_loader) - 1):
            print(
                f"Epoch: [{epoch + 1}][{step}/{len(train_loader)}] "
                f"Elapsed {timeSince(start, float(step + 1) / len(train_loader)):s} "
                f"Loss: {losses.avg:.4f} "
            )

    return losses.avg

## 評価 補助関数

In [25]:
def valid_fn(valid_loader, model, criterion, device):
    start = end = time.time()
    losses = AverageMeter()

    # switch to evaluation mode
    model.eval()
    preds = []

    for step, (input_ids, attention_mask, labels) in enumerate(valid_loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        # compute loss
        with torch.no_grad():
            y_preds = model(input_ids, attention_mask)

        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)

        # record score
        preds.append(y_preds.to("cpu").numpy())

        if step % 100 == 0 or step == (len(valid_loader) - 1):
            print(
                f"EVAL: [{step}/{len(valid_loader)}] "
                f"Elapsed {timeSince(start, float(step + 1) / len(valid_loader)):s} "
                f"Loss: {losses.avg:.4f} "
            )

    predictions = np.concatenate(preds)
    return losses.avg, predictions

## 推論関数

In [26]:
def inference():
    predictions = []

    test_dataset = BaseDataset(test, CFG.model, include_labels=False)
    test_loader = DataLoader(
        test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers, pin_memory=True
    )

    for fold in range(CFG.n_splits): #5
    #for fold in [1,3,4,5]: #5
        LOGGER.info(f"========== model: bert-base-uncased fold: {fold} inference ==========")
        model = BaseModel(CFG.model)
        model.to(device)
        model.load_state_dict(torch.load(OUTPUT_DIR + f"{CFG.version}_fold{fold}_best.pth")["model"])
        model.eval()
        preds = []
        
        for i, (input_ids, attention_mask) in tqdm(enumerate(test_loader), total=len(test_loader)):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            with torch.no_grad():
                y_preds = model(input_ids, attention_mask)
            preds.append(y_preds.to("cpu").numpy())
            
        preds = np.concatenate(preds)
        predictions.append(preds)
        
    predictions = np.mean(predictions, axis=0)

    return predictions

## 学習

In [27]:
def train_loop(train, fold):

    LOGGER.info(f"=*========= fold: {fold} training ==========")
    
    # ====================================================
    # Data Loader
    # ====================================================
    trn_idx = train[train["fold"] != fold].index
    val_idx = train[train["fold"] == fold].index
    
    train_folds = train.loc[trn_idx].reset_index(drop=True)
    valid_folds = train.loc[val_idx].reset_index(drop=True)
    
    train_dataset = BaseDataset(train_folds, CFG.model)
    valid_dataset = BaseDataset(valid_folds, CFG.model)
    #print("DataLoader")
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.batch_size, #16
        shuffle=True,
        num_workers=CFG.num_workers, #4, 
        pin_memory=True,
        drop_last=True,
    )
    
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=CFG.batch_size, #16
        shuffle=False,
        num_workers=CFG.num_workers, #4,
        pin_memory=True,
        drop_last=False,
    )
    #print("Model")
    # ====================================================
    # Model
    # ====================================================
    model = BaseModel(CFG.model)
    model.to(device)

    optimizer = T.AdamW(model.parameters(), lr=2e-5)

    criterion = nn.BCELoss()
    #criterion = nn.BCEWithLogitsLoss()
    # ====================================================
    # Loop
    # ====================================================
    best_score = -1
    best_loss = np.inf
    #print("Loop")
    for epoch in range(CFG.epochs):
        start_time = time.time()
        
        # train
        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, device)
        
        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
        valid_labels = valid_folds[TARGET].values
        
        # scoring
        score = fbeta_score(valid_labels, np.where(preds < init_border, 0, 1), beta=7.0)
        LOGGER.info(f"score1 = {score}, thresh={init_border}")
        
        border = opt_fbeta_threshold(valid_labels, preds) ##最適化
        
        score = fbeta_score(valid_labels, np.where(preds < border, 0, 1), beta=7.0)
        LOGGER.info(f"score2 = {score},  thresh={border}")
        
        elapsed = time.time() - start_time
        LOGGER.info(
            f"Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s"
        )
        LOGGER.info(f"Epoch {epoch+1} - Score: {score}")

        #if score >= best_score: ##
        if avg_val_loss <= best_loss:
        #if True:
            best_loss = avg_val_loss
            best_score = score
            LOGGER.info(f"Epoch {epoch+1} - Save Best Score: {best_score:.4f} - Best Loss: {best_loss:.4f} Model")
            torch.save(
                {"model": model.state_dict(), "preds": preds}, OUTPUT_DIR + f"{CFG.version}_fold{fold}_best.pth"
            )

    check_point = torch.load(OUTPUT_DIR + f"{CFG.version}_fold{fold}_best.pth")

    valid_folds["preds"] = check_point["preds"]

    return valid_folds

## メイン

In [28]:
def get_result(result_df):
    preds = result_df["preds"].values
    labels = result_df["judgement"].values
    
    score = fbeta_score(labels, np.where(preds < border, 0, 1), beta=7.0)
    LOGGER.info(f"Score1: {score:<.5f}")
    
    best_border = opt_fbeta_threshold(labels, preds)
    LOGGER.info(f"best border: {best_border:<.5f}")
    
    score = fbeta_score(labels, np.where(preds < best_border, 0, 1), beta=7.0)
    LOGGER.info(f"Score2: {score:<.5f}")
    
    return score, best_border

In [29]:
from scipy.optimize import minimize, minimize_scalar
def opt_fbeta_threshold(y_true, y_pred):
    """fbeta score計算時のthresholdを最適化"""
    def opt_(x): 
        return -fbeta_score(y_true, y_pred >= x, beta=7)
    
    #result = minimize(opt_, x0=np.array([0.1]), method='Powell')
    result = minimize_scalar(opt_, bounds=(0, 0.5), method='bounded')
    
    best_threshold = result['x'].item()
    return best_threshold

In [30]:
def visualize_confusion_matrix(
        y_true,
        pred_label,
        height=.6,
        labels=None):
    """混合行列をプロット 
    (https://www.guruguru.science/competitions/11/discussions/2fb11851-67d0-4e96-a4b1-5629b944f363/)"""
    
    conf = confusion_matrix(y_true=y_true,
                            y_pred=pred_label,
                            normalize='true')

    n_labels = len(conf)
    size = n_labels * height
    fig, ax = plt.subplots(figsize=(size * 4, size * 3))
    sns.heatmap(conf, cmap='Blues', ax=ax, annot=True, fmt='.2f')
    ax.set_ylabel('Label')
    ax.set_xlabel('Predict')

    if labels is not None:
        ax.set_yticklabels(labels)
        ax.set_xticklabels(labels)
        ax.tick_params('y', labelrotation=0)
        ax.tick_params('x', labelrotation=90)

    plt.show()
    return fig

In [31]:
##main

oof_df = pd.DataFrame()
for fold in range(CFG.n_splits): 
    _oof_df = train_loop(train, fold)
    oof_df = pd.concat([oof_df, _oof_df])
    LOGGER.info(f"========== fold: {fold} result ==========")
    get_result(_oof_df)
        
# CV result
LOGGER.info(f"========== CV ==========")
score, best_border = get_result(oof_df)
    
# Save OOF result
oof_df.to_csv(OUTPUT_DIR + f"oof_df_{CFG.version}_{score:<.5f}.csv", index=False)


=*========= fold: 0 training ==========
Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceCl

Epoch: [1][0/4343] Elapsed 0m 0s (remain 57m 28s) Loss: 0.6753 
Epoch: [1][100/4343] Elapsed 0m 18s (remain 13m 1s) Loss: 0.1201 
Epoch: [1][200/4343] Elapsed 0m 36s (remain 12m 31s) Loss: 0.1145 
Epoch: [1][300/4343] Elapsed 0m 54s (remain 12m 14s) Loss: 0.1187 
Epoch: [1][400/4343] Elapsed 1m 12s (remain 11m 54s) Loss: 0.1097 
Epoch: [1][500/4343] Elapsed 1m 30s (remain 11m 37s) Loss: 0.1036 
Epoch: [1][600/4343] Elapsed 1m 49s (remain 11m 20s) Loss: 0.1004 
Epoch: [1][700/4343] Elapsed 2m 7s (remain 11m 1s) Loss: 0.0946 
Epoch: [1][800/4343] Elapsed 2m 25s (remain 10m 41s) Loss: 0.0919 
Epoch: [1][900/4343] Elapsed 2m 43s (remain 10m 22s) Loss: 0.0923 
Epoch: [1][1000/4343] Elapsed 3m 0s (remain 10m 4s) Loss: 0.0898 
Epoch: [1][1100/4343] Elapsed 3m 18s (remain 9m 45s) Loss: 0.0862 
Epoch: [1][1200/4343] Elapsed 3m 36s (remain 9m 26s) Loss: 0.0863 
Epoch: [1][1300/4343] Elapsed 3m 54s (remain 9m 8s) Loss: 0.0846 
Epoch: [1][1400/4343] Elapsed 4m 12s (remain 8m 50s) Loss: 0.0847 
Epo

score1 = 0.7509881422924901, thresh=0.023282372444280715
score2 = 0.7734806629834254,  thresh=0.017227239527402384
Epoch 1 - avg_train_loss: 0.0620  avg_val_loss: 0.0614  time: 836s
Epoch 1 - Score: 0.7734806629834254
Epoch 1 - Save Best Score: 0.7735 - Best Loss: 0.0614 Model


EVAL: [1085/1086] Elapsed 0m 57s (remain 0m 0s) Loss: 0.0614 
Epoch: [2][0/4343] Elapsed 0m 0s (remain 25m 11s) Loss: 0.0014 
Epoch: [2][100/4343] Elapsed 0m 18s (remain 12m 46s) Loss: 0.0364 
Epoch: [2][200/4343] Elapsed 0m 36s (remain 12m 24s) Loss: 0.0450 
Epoch: [2][300/4343] Elapsed 0m 54s (remain 12m 5s) Loss: 0.0417 
Epoch: [2][400/4343] Elapsed 1m 11s (remain 11m 47s) Loss: 0.0407 
Epoch: [2][500/4343] Elapsed 1m 29s (remain 11m 28s) Loss: 0.0428 
Epoch: [2][600/4343] Elapsed 1m 47s (remain 11m 10s) Loss: 0.0421 
Epoch: [2][700/4343] Elapsed 2m 5s (remain 10m 52s) Loss: 0.0411 
Epoch: [2][800/4343] Elapsed 2m 23s (remain 10m 34s) Loss: 0.0433 
Epoch: [2][900/4343] Elapsed 2m 41s (remain 10m 16s) Loss: 0.0431 
Epoch: [2][1000/4343] Elapsed 2m 59s (remain 9m 58s) Loss: 0.0437 
Epoch: [2][1100/4343] Elapsed 3m 17s (remain 9m 40s) Loss: 0.0421 
Epoch: [2][1200/4343] Elapsed 3m 34s (remain 9m 22s) Loss: 0.0443 
Epoch: [2][1300/4343] Elapsed 3m 52s (remain 9m 4s) Loss: 0.0444 
Epoch:

score1 = 0.8250825082508251, thresh=0.023282372444280715
score2 = 0.9033353922174182,  thresh=0.0038147669210738446
Epoch 2 - avg_train_loss: 0.0398  avg_val_loss: 0.0441  time: 834s
Epoch 2 - Score: 0.9033353922174182
Epoch 2 - Save Best Score: 0.9033 - Best Loss: 0.0441 Model


...

score1 = 0.8348240906380441, thresh=0.023282372444280715
score2 = 0.8159564823209429,  thresh=0.10400718253966176
Epoch 3 - avg_train_loss: 0.0306  avg_val_loss: 0.0551  time: 834s
Epoch 3 - Score: 0.8159564823209429


EVAL: [1085/1086] Elapsed 0m 56s (remain 0m 0s) Loss: 0.0551 
Epoch: [4][0/4343] Elapsed 0m 0s (remain 25m 52s) Loss: 0.0012 
Epoch: [4][100/4343] Elapsed 0m 18s (remain 12m 47s) Loss: 0.0232 
Epoch: [4][200/4343] Elapsed 0m 36s (remain 12m 24s) Loss: 0.0141 
Epoch: [4][300/4343] Elapsed 0m 54s (remain 12m 5s) Loss: 0.0188 
Epoch: [4][400/4343] Elapsed 1m 11s (remain 11m 46s) Loss: 0.0155 
Epoch: [4][500/4343] Elapsed 1m 29s (remain 11m 28s) Loss: 0.0200 
Epoch: [4][600/4343] Elapsed 1m 47s (remain 11m 10s) Loss: 0.0234 
Epoch: [4][700/4343] Elapsed 2m 5s (remain 10m 52s) Loss: 0.0215 
Epoch: [4][800/4343] Elapsed 2m 23s (remain 10m 34s) Loss: 0.0218 
Epoch: [4][900/4343] Elapsed 2m 41s (remain 10m 16s) Loss: 0.0214 
Epoch: [4][1000/4343] Elapsed 2m 59s (remain 9m 58s) Loss: 0.0222 
Epoch: [4][1100/4343] Elapsed 3m 16s (remain 9m 40s) Loss: 0.0212 
Epoch: [4][1200/4343] Elapsed 3m 34s (remain 9m 22s) Loss: 0.0217 
Epoch: [4][1300/4343] Elapsed 3m 52s (remain 9m 4s) Loss: 0.0216 
Epoch:

score1 = 0.5454389280192407, thresh=0.023282372444280715
score2 = 0.0,  thresh=0.49999553788108364
Epoch 4 - avg_train_loss: 0.0384  avg_val_loss: 0.1136  time: 834s
Epoch 4 - Score: 0.0


EVAL: [1085/1086] Elapsed 0m 56s (remain 0m 0s) Loss: 0.1136 


========== fold: 4 result ==========
Score1: 0.85100
best border: 0.05466
Score2: 0.87850
========== CV ==========
Score1: 0.86591
best border: 0.00706
Score2: 0.89477


In [32]:
border, best_border

(0.023282372444280715, 0.0070590010853141885)

In [33]:
# Inference
predictions = inference()

========== model: bert-base-uncased fold: 0 inference ==========
Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model

  0%|          | 0/8167 [00:00<?, ?it/s]

========== model: bert-base-uncased fold: 1 inference ==========
Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model

  0%|          | 0/8167 [00:00<?, ?it/s]

========== model: bert-base-uncased fold: 2 inference ==========
Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model

  0%|          | 0/8167 [00:00<?, ?it/s]

========== model: bert-base-uncased fold: 3 inference ==========
Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model

  0%|          | 0/8167 [00:00<?, ?it/s]

========== model: bert-base-uncased fold: 4 inference ==========
Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model

  0%|          | 0/8167 [00:00<?, ?it/s]

In [34]:
# for ensemble
test["judgement"] = predictions
test.to_csv(OUTPUT_DIR + f"predictions_{CFG.version}.csv", index=False, header=False)

In [38]:
#border2 = 0.0070590010853141885
# submission
predictions2 = np.where(predictions < border, 0, 1)
sub["judgement"] = predictions2
sub.to_csv(OUTPUT_DIR + f"submission_{CFG.version}-2.csv", index=False, header=False)